In [2]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from mne.viz import plot_alignment, set_3d_view
# from pathlib import Path


## Reading edf files

In [3]:
patient_id = 1
DATASET_PATH = os.path.join(os.getcwd(), 'data', 'chb01-summary.txt') #ONLY FOR TESTING
patient_id_str = str(patient_id).zfill(2)

all_file_path = glob(f'chb{patient_id_str}/*.edf')
print(len(all_file_path))

4


In [4]:
raw = mne.io.read_raw_edf(all_file_path[0])
print(raw.get_channel_types())
print(raw.info)
print(raw.ch_names)
print(raw.get_data().shape)

Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
<Info | 7 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 13:43:04 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


/tmp/ipykernel_74466/2924192598.py:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(all_file_path[0])


(23, 921600)


In [17]:
def read_data_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration=10, overlap=1)
    epochs_array = epochs.get_data()
    return epochs, epochs_array
# def read_data_seizures(file_path):
#     ref_file = open(file_path, 'r')
#     array=epochs.get_data()
#     return array

def read_edf_to_raw(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    raw.filter(l_freq=0.5, h_freq=45)
    return raw

In [6]:
%%capture

# Shape ()
epochs_data_list = [read_data_edf(i)[1] for i in all_file_path]
epochs_list = [read_data_edf(i)[0] for i in all_file_path]


In [81]:
epochs_data_array = np.array(epochs_data_list)
epochs_data_array.shape

(4, 399, 23, 2560)

In [15]:
# Each file has a representant epochs objects that contains all the epochs.  
print(epochs_list[0].metadata)

None


In [ ]:
# merge the files on one array 
# shape: (N_files, 399 epochs, 23 channels, 2560 values) ->  (N_files*399 epochs, 23 channels, 2560 values)
data_array = np.vstack(data_epochs)
data_array.shape

## Extract features

In [41]:
########! Features extraction !#####
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np.argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x, axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return [mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmaxim(x), 
                           argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)]

In [52]:
features = []
for d in data_array:
    features.append(concatenate_features(d))
    
features_array = np.array(features)
features_array.shape

(1596, 276)

## Visualize features per channel and per epoch

In [63]:
# get features of specific channel
def features_by_channel(features_array):
    features_channel = []
    MAX_CHANNELS = 23
    for i in range(MAX_CHANNELS):
        array = features_array[:, i*12:(i+1)*12];
        features_channel.append(pd.DataFrame(array, columns=['Mean', 'Std', 'Ptp', 'Var', 'Minimo', 'Maxim', 'Arg Max', 'Arg Min', 'RMS', 'ABS DIFF', 'skewness', 'kurtosis']))
    return features_channel

features_channel = features_by_channel(features_array)
print('Info for channel 23: ')

# Shape of features_channel=(23, 1596, 12)
features_channel[22].shape

Info for channel 23: 


(1596, 12)

## Labelling

In [24]:
# patient_folder = Path.cwd()/'chbmit'/'chb{}'.format(str(patient_id).zfill(2))
# DATASET_PATH = 
DATASET_PATH = os.getcwd() #ONLY FOR TESTING

patient_id = 1
patient_id_str = str(patient_id).zfill(2)

patient_folder = os.path.join(DATASET_PATH, f'chb{patient_id_str}')
print(f"Serching for patient {patient_id_str} files at {patient_folder} ...")
patient_files = glob(os.path.join(patient_folder, f'*.edf'))
print(f"Found {len(patient_files)} files")
summary_path = os.path.join(patient_folder, f'chb{patient_id_str}-summary.txt')

Serching for patient 01 files at /home/guisoares/soares_repo/SCC0276-Machine-Learning/chb01 ...
Found 4 files


In [49]:
## READ SUMMARY (TODO: automate this)
seizures_dict = {"chb01_03": [[2996, 3036]],
                "chb01_04": [[1467, 1494]],
                "chb01_15": [[1732, 1772]],
                "chb01_16": [[1015, 1066]],
                "chb01_18": [[1720, 1810]],
                "chb01_21": [[327, 420]],
                "chb01_26": [[1862, 1963]],
                
                
                "chb02_16": [[130, 212]],
                "chb05_06": [[417, 532]], 
                "chb05_13": [[1086, 1196]],
                "chb05_16": [[2317, 2413]], 
                "chb05_17": [[2451, 2571]],
                "chb05_22": [[2348, 2465]],
                "chb08_02": [[2670, 2841]], 
                "chb08_05": [[2856, 3046]],
                "chb08_11": [[2988, 3211]], 
                "chb08_13": [[2417, 2577]],
                "chb08_21": [[2083, 2347]]}

curr_time = 0
epoch_time = 10
overlap = 5

# Divide into epochs
labels = []
for file_path in patient_files:
    # get filename: chbxx_xx.edf
    filename = os.path.split(file_path)[1]

    # remove .edf staying only chbxx_xx
    filename = os.path.splitext(filename)[0]
    
    # read raw
    raw = read_edf_to_raw(file_path)

    curr_time = 0
    epoch_features = []
    while curr_time <= max(raw.times) + 0.01 - epoch_time:  # max(raw.times) = 3600
        
        start_time = curr_time 
        if start_time < 0.:
            start_time = 0.
        end_time = curr_time + epoch_time - overlap
        start, stop = raw.time_as_index([start_time, end_time])
        temp = raw[:, start:stop][0]

        # start time as ID
        # features.append(start_time)

        # features
        for i in range(23):
            epoch_features.append(concatenate_features(temp[i]))

        # seizure flag for y
        aux = []
        if filename in seizures_dict:  # if file has seizure
            for seizure in seizures_dict[filename]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    aux.append(1)
                elif start_time + epoch_time > seizure[0] and start_time + epoch_time < seizure[1]:
                    aux.append(1)
                else:
                    aux.append(0)
        else:    
            aux.append(0)

        if 1 in aux:
            epoch_features.append(1)
        else:
            epoch_features.append(0)

        labels.append(epoch_features)

        curr_time = curr_time + epoch_time - overlap  
        print("Section ", str(len(labels)), "; start: ", start, " ; stop: ", stop)

Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_74466/4000768790.py:14: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

Section  1 ; start:  0  ; stop:  1280
Section  2 ; start:  1280  ; stop:  2560
Section  3 ; start:  2560  ; stop:  3840
Section  4 ; start:  3840  ; stop:  5120
Section  5 ; start:  5120  ; stop:  6400
Section  6 ; start:  6400  ; stop:  7680
Section  7 ; start:  7680  ; stop:  8960
Section  8 ; start:  8960  ; stop:  102

KeyboardInterrupt: 

In [50]:
labels = np.array(labels)

/tmp/ipykernel_74466/796522320.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array(labels)


In [51]:
labels.shape

(53, 1272)

In [48]:
labels[0]

[[3.475458005073624e-06,
  2.137830104024441e-05,
  0.00011792420274864188,
  4.5703175536731527e-10,
  -5.575658280806377e-05,
  6.216761994057811e-05,
  54,
  587,
  2.1658960356220832e-05,
  0.004350763437071228,
  -0.07710882684457487,
  -0.40202232274619076],
 [-2.755508384979668e-06,
  2.280666146254042e-05,
  0.00013066515918841085,
  5.201438070669263e-10,
  -5.745817183576676e-05,
  7.32069873526441e-05,
  742,
  203,
  2.297251909405278e-05,
  0.0030465608590553627,
  0.10861274065499368,
  -0.12853389870278686],
 [1.713126995090508e-06,
  1.970957116995924e-05,
  0.0001247127487517994,
  3.884671957036885e-10,
  -6.474819625768232e-05,
  5.9964552494117104e-05,
  604,
  728,
  1.9783882323876582e-05,
  0.002787264333353323,
  -0.0530276060005069,
  0.4404976700728427],
 [3.551186632649331e-08,
  1.594655405752321e-05,
  0.0001060450501462994,
  2.5429258630950993e-10,
  -5.723982947111153e-05,
  4.880522067518787e-05,
  586,
  525,
  1.5946593598701886e-05,
  0.0033466999483